## Data Preprocessing

In [2]:
import numpy as np
import pandas as pd
import sys


sys.path.append('helper.py')
import helper as hp

import importlib 
importlib.reload(hp)

Helper code loaded
Helper code loaded


<module 'helper' from 'c:\\Users\\victo\\OneDrive\\Documents\\UPTP - Documents\\Year 4\\Machine Learning\\Machine Learning - PowerConsumptionPrediction Project\\PowerConsumtionPrediction\\Pre Processing\\helper.py'>

In [51]:
import re

def dms_to_decimal(dms_str):
    """Convert DMS string like '25°04′34.42″N' to decimal degrees."""
    pattern = r"(\d+)°(\d+)'([\d.]+)\"?([NSEW])"
    match = re.match(pattern, dms_str.strip().replace("′", "'").replace("″", '"'))
    
    if not match:
        raise ValueError(f"Invalid DMS format: {dms_str}")
    
    degrees, minutes, seconds, direction = match.groups()
    decimal = int(degrees) + int(minutes) / 60 + float(seconds) / 3600
    
    if direction in ['S', 'W']:
        decimal *= -1
    
    return decimal

def split_lat_lon(latlon_str):
    """Split '25°04′34.42″N 121°25′08.59″E' into separate lat and lon decimal values."""
    parts = latlon_str.split()
    lat = dms_to_decimal(parts[0])
    lon = dms_to_decimal(parts[1])
    return lat, lon


## Generate clean temperature data
This code generates a cleaner csv with monthly average temperatures for each station

In [86]:
import pandas as pd

station_list = pd.read_csv("../Raw Data/Meteorological Station List.csv")

# Store parsed data
stations_data = []

for _, row in station_list.iterrows():
    station = row['Meteorological Station']
    latlon = row['LatitudeLongitude']
    filename = row['File Name']
    
    try:
        # Convert DMS to decimal
        lat, lon = split_lat_lon(latlon)
        
        temp_data = pd.read_csv(f"../Raw Data/Meteorological Data/{filename}.csv", skiprows=5)
        average_temps = temp_data.iloc[-1, 1:-1]

        record = {
            'Station': station,
            'Latitude': lat,
            'Longitude': lon
        }

        for m in range(0, 12):
            record[str(m + 1)] = average_temps[str(m+1)]
        
        stations_data.append(record)

    except Exception as e:
        print(f"Failed to parse row: {station}, error: {e}")

stations_df = pd.DataFrame(stations_data)

# Export
stations_df.to_csv("../Clean Data/Meteorological Station Data Clean.csv", index=False)

## Generate Clean Monthly Consumption with Climate Type

In [ ]:
consumption_original = pd.read_csv("../Clean Data/monthly consumption per station.csv")
print(consumption_original.head(1))

  Station      gantry   Latitude   Longitude  Lane Count  Month  \
0      后里  01F-104.5N  24.727131  120.951897         3.0      5   

   Estimated Monthly Consumption (kWh)  
0                               1738.5  
